Part B: Create a notebook called linear_regression<br>
<br>
Python: Initial data prep section.  Read, clean and create sets.<br>
Markdown: Pick an initial set of features for X and the target feature y.  Explain <br>why you made this choice.<br>
Python: Do a linear regression on X and y.  Compute metrics.<br>
Markdown: Comment on the results.<br>
Python: See if you can do better.<br>
Python: Do a final evaluation with the test set.<br>
Markdown: Look at the parameters you found and discuss what you have learned.<br>

In [13]:
#bringing in, cleaning, and merging datasets
import pandas as pd
#import the csv files
weather = pd.read_csv("seattle-weather.csv")
reports = pd.read_csv("SPD_Reports.csv")


#remove data that will be unused
reports.drop(columns=["Offense Description", "Block", "District","Beat","2000 Census Tract","Longitude","Latitude","Offense Start Date","Offense End Date"],inplace=True)

#fix data types
reports["Report Date"] = pd.to_datetime(reports["Report Date"])
weather["date"] = pd.to_datetime(weather["date"])

weather_mapping = {'drizzle': 0, 'rain': 1, 'sun': 2, 'snow': 3, 'fog': 4}
weather["weather"] = weather["weather"].map(weather_mapping)

# Define the date range covered by data sets
start_date = pd.to_datetime("2014-01-17")
end_date = pd.to_datetime("2015-12-31")

# Filter out unusable records
reports_filtered = reports[(reports["Report Date"] >= start_date) & (reports["Report Date"] <= end_date)]
weather_filtered = weather[(weather["date"] >= start_date) & (weather["date"] <= end_date)]

#merge datasets
merged_data = pd.merge(weather_filtered, reports_filtered.groupby('Report Date').size().reset_index(name='Offense Count'), left_on="date", right_on="Report Date", how="left")


In [14]:
#Creating the model
import numpy as np
def fractional_split(data_set, test_fraction=0.2, seed=41):# 41 pretty good seed
    data_count = len(data_set)
    test_count = int(test_fraction*data_count)
    
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(data_count)
    
    # Use the front of the shuffled list as the test set
    # Use the bask of the shuffled list as the training set
    test_indices = shuffled_indices[:test_count]
    train_indices = shuffled_indices[test_count:]
    
    return data_set.iloc[train_indices], data_set.iloc[test_indices]

# Replace null values in the "Offense Count" column with 0
merged_data['Offense Count'].fillna(0, inplace=True)

train_set, test_set = fractional_split(merged_data)

# Make a copy of the training set
working_set = train_set.copy()
use =["precipitation", "temp_max", "temp_min", "wind","weather"]
find = "Offense Count"

from sklearn.linear_model import LinearRegression
reg = LinearRegression()

X = working_set[use]
Y = working_set[find]
reg.fit(X, Y)

LinearRegression()

I chose to train the data set using <br>
"precipitation", "temp_max", "temp_min", "wind","weather"<br>
Because all of these attributes reflect the overall type of day.<br>
I thought there might be a correlation between the condition of the day<br>
and crimes because I figured people wouldn't be out as much on days with<br>
worse conditions. <br>
I also didn't realize that using two different data sets was not a <br>
requirement. But hey, now I know how to merge data sets and stuff. 

In [15]:
#Testing the model
X_test = test_set[use]
Y_test = test_set[find]
Y_pred = reg.predict(X_test)

# Calculate Mean Squared Error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, Y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.09836797688518946


The model had a Mean Squared Error value of .09. If you compare this to <br>
The model in the initial_exploration notebook that didn't predict and only<br>
predicted 0, had a Mean Squared Error value of .22. This means that this<br>
model is an average of .13 closer to the actual value than the unpredictive<br>
model. <br>
<br>
I have tested this model a fair bit and have found 41 to be the best seed.<br>
42 and 38 are particularly bad seeds, being almost as inaccurate as the
unpredictive model.<br>
In addition to that when I first created it i didn't have the "weather" <br>
attribute included in the predictive model. By adding it the model increased<br>
accuracy. 